In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import yaml

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

from statsmodels.stats.multitest import multipletests
from statsmodels.api import Logit,add_constant

from tqdm.notebook import trange
import joblib

In [2]:
%matplotlib inline
%load_ext watermark

In [3]:
# set a fixed random state for reproduceability random state from random.org
import random
random.seed(0x8ee4046d6bd8e6ef)
np.random.seed(random.randint(0, 0x1<<31))

In [4]:
with open('../data/gene-names.yaml') as in_fd:
    name_map = yaml.load(in_fd, yaml.FullLoader)
    
name_map.update({
    'sample' : 'sample',
    'accession' : 'accession',
    'age-at-diag' : 'age-at-diag',
    'death' : 'death',
    'over-all-survival' : 'over-all-survival',
    'event-free-survival' : 'event-free-survival'
})

expression_data = pd.read_csv('../data/GSE142102.csv.xz')

expression_data.rename(
    inplace=True,
    columns={
        k : name_map.get(k, k)
            for k in expression_data.columns
    }
)
del name_map
expression_data.head()

,sample,accession,age-at-diag,death,batch,DDX11L1,MIR1302-11,OR4F5,LOC100132062,LINC00266-1,...,FP2234,CDC14B.1,ZNF322.1,LOC100128657,LOC100652907,RBSG2,LOC401561.1,LOC651337,TXLNG2P.1,TEKT4P2.1
0,Tumor specimen 191,GSM4220622,78,True,2,0.337813,-0.381713,0.853635,0.405675,1.017472,...,1.515277,-0.153337,-0.183246,0.244214,-1.844791,2.089169,-0.802096,-0.331706,1.513253,-0.703120
1,Tumor specimen 001,GSM4220623,37,True,1,-0.360069,-0.930706,-0.585504,1.188569,0.495601,...,0.227174,0.413321,1.763495,-0.508197,1.230847,0.886788,-1.428903,-0.111950,-0.321647,-0.905078
2,Tumor specimen 002,GSM4220624,56,True,1,-0.618881,-1.020463,-0.441207,0.991068,1.340864,...,-0.810626,0.637207,-0.383648,-1.319821,-0.695103,-0.448736,-1.193096,0.365682,0.525380,-0.402475
3,Tumor specimen 003,GSM4220625,60,True,1,-0.527342,-0.403463,1.886248,0.821856,0.448809,...,-0.449835,-0.802237,0.127335,0.317466,0.054231,-0.548393,-1.384320,0.147551,-1.308738,0.651851
4,Tumor specimen 081,GSM4220626,47,False,2,0.366954,-1.965376,1.318138,-1.884689,-1.350717,...,3.127458,0.212165,0.252226,1.968311,-1.067728,0.552572,0.783284,1.866398,-0.357060,1.182845


In [5]:
clf = joblib.load('../data/classifier.joblib')
feature_names = clf.feature_names_in_
missing_feature_names = [f for f in feature_names if f not in expression_data.columns]
print('missing genes', *missing_feature_names)

missing genes PRR34 MIR3667HG CCL4L1 CHURC1 CRHR1 BHLHE22-AS1 XRCC6P5 POLR2J3 Prss45 Smc1b SSPOP


In [6]:
metadata = ['death', 'age-at-diag']
genes = [f for f in feature_names if f in expression_data.columns]
print(expression_data['death'].value_counts())
expression_data[metadata + genes].describe()

False    123
True      87
Name: death, dtype: int64


,age-at-diag,ADAM19,ADH5,BTNL8,DHRS4-AS1,PLGRKT,CALHM1,CAT,CRISPLD1,CROCCP2,...,DYNLT5,TMEM177,TTPA,TUBB8,TYW1B,UCN2,WEE2,ZC3H8,ZFP64,ZNF629
count,210.000000,210.000000,210.000000,210.000000,210.000000,210.000000,210.000000,210.000000,210.000000,210.000000,...,210.000000,210.000000,210.000000,210.000000,210.000000,210.000000,210.000000,210.000000,210.000000,210.000000
mean,54.504762,0.026873,0.018503,-0.013675,-0.036763,0.019537,-0.011113,0.017264,0.007628,0.003843,...,-0.018816,0.015734,-0.012086,-0.026014,0.014285,0.011823,-0.029558,0.035399,-0.033333,-0.035733
std,13.231920,1.021492,0.995551,1.002823,0.983658,0.985664,1.017656,1.003582,1.026800,1.012565,...,0.994656,1.014897,0.973293,0.988814,0.973717,1.004662,0.989118,1.012861,0.970265,0.994781
min,24.000000,-1.991166,-2.534890,-1.848248,-1.933314,-2.708555,-1.849431,-2.520589,-1.513089,-2.594508,...,-1.988217,-2.331462,-1.875599,-2.138013,-2.890469,-1.693183,-1.933231,-2.041745,-2.395663,-3.136979
25%,46.000000,-0.604571,-0.676300,-0.698228,-0.699895,-0.690407,-0.791164,-0.608478,-0.639757,-0.745935,...,-0.723149,-0.772516,-0.693797,-0.728980,-0.587724,-0.672991,-0.768302,-0.713039,-0.692100,-0.743460
50%,55.000000,-0.149162,-0.016250,-0.133909,-0.201426,-0.037331,-0.213171,-0.089968,-0.320279,0.040982,...,-0.158090,-0.119495,-0.132960,-0.183614,0.025097,-0.112670,-0.171316,-0.081661,-0.110778,-0.064770
75%,62.000000,0.561636,0.563396,0.695192,0.501778,0.721149,0.644900,0.468376,0.262778,0.591509,...,0.482464,0.555268,0.442185,0.560665,0.576848,0.532765,0.574527,0.668077,0.375716,0.614235
max,90.000000,5.400037,2.750085,3.185150,5.572410,3.380768,4.551839,4.100596,4.714072,2.871503,...,4.154200,3.078168,3.785314,2.724531,2.665864,4.089165,3.184620,3.289636,3.459694,2.962887


In [72]:
clf = Logit(
    expression_data['death'],
    add_constant(expression_data[['age-at-diag'] + genes])
)
res = clf.fit()
res.summary()

Optimization terminated successfully.
         Current function value: 0.608819
         Iterations 5


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                  death   No. Observations:                  210
Model:                          Logit   Df Residuals:                      187
Method:                           MLE   Df Model:                           22
Date:                Fri, 11 Mar 2022   Pseudo R-squ.:                  0.1025
Time:                        02:12:20   Log-Likelihood:                -127.85
converged:                       True   LL-Null:                       -142.46
Covariance Type:            nonrobust   LLR p-value:                    0.1388
===============================================================================
                  coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------
const          -2.2867      0.715     -3.200      0.001      -3.687      -0.886
age-at-diag     0.0349      0.013      2.763      0.006       0.010       0.060
RETN           -0.0167      0.205     -0.081      0.935      -0.419       0.386
HIP1R           0.2140      0.156      1.373      0.170      -0.092       0.520
DDX51           0.0960      0.176      0.545      0.586      -0.249       0.441
LRRC37A2       -0.3082      0.247     -1.249      0.212      -0.792       0.175
PWP2            0.0182      0.163      0.112      0.911      -0.300       0.337
CROCCP2         0.0067      0.168      0.040      0.968      -0.322       0.336
ELMO3           0.3572      0.167      2.135      0.033       0.029       0.685
PRKY           -0.0291      0.168     -0.173      0.862      -0.358       0.300
FLJ26850        0.1334      0.168      0.793      0.428      -0.196       0.463
KIAA1324L       0.1521      0.162      0.941      0.347      -0.165       0.469
ZFP64           0.0711      0.172      0.414      0.679      -0.266       0.408
DHRS4-AS1       0.1274      0.171      0.745      0.456      -0.208       0.462
PPIL3          -0.1586      0.179     -0.885      0.376      -0.510       0.193
KLK10          -0.1940      0.160     -1.210      0.226      -0.508       0.120
ASH1L-AS1      -0.0619      0.166     -0.373      0.709      -0.386       0.263
GUSBP3         -0.0014      0.161     -0.008      0.993      -0.317       0.314
GPR17          -0.3196      0.200     -1.599      0.110      -0.711       0.072
IQCK            0.1445      0.180      0.804      0.421      -0.208       0.497
DNASE1L1        0.1089      0.168      0.649      0.516      -0.220       0.437
HEXIM1          0.0840      0.177      0.475      0.634      -0.262       0.430
FOXQ1          -0.1804      0.171     -1.054      0.292      -0.516       0.155
===============================================================================
"""

In [74]:
res.wald_test_terms()

/home/seth/miniconda3/envs/condaenv/lib/python3.9/site-packages/statsmodels/base/model.py:1889: FutureWarning: The behavior of wald_test will change after 0.14 to returning scalar test statistic values. To get the future behavior now, set scalar to True. To silence this message while retaining the legacy behavior, set scalar to False.
  warnings.warn(


<class 'statsmodels.stats.contrast.WaldTestResults'>
                                  chi2                 P>chi2  df constraint
const           [[10.242088887820664]]  0.0013727204885737201              1
age-at-diag      [[7.633190779484013]]  0.0057303836894150315              1
RETN          [[0.006607304940830337]]     0.9352150400151353              1
HIP1R           [[1.8844478367613196]]    0.16982948203419024              1
DDX51           [[0.2970112176455453]]     0.5857622141286783              1
LRRC37A2         [[1.560732073181946]]    0.21155806287408724              1
PWP2          [[0.012548160420981221]]     0.9108086811081365              1
CROCCP2      [[0.0015731629291549332]]     0.9683617061003806              1
ELMO3            [[4.558714514158914]]    0.03275164730198191              1
PRKY           [[0.03008566424430818]]     0.8622960004477606              1
FLJ26850        [[0.6291460723963828]]     0.4276687132176493              1
KIAA1324L        [[0.88

In [75]:
clf = Logit(
    expression_data['death'],
    add_constant(expression_data[['age-at-diag'] + ['ELMO3']])
)
res = clf.fit()
res.summary()

Optimization terminated successfully.
         Current function value: 0.643687
         Iterations 5


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                  death   No. Observations:                  210
Model:                          Logit   Df Residuals:                      207
Method:                           MLE   Df Model:                            2
Date:                Fri, 11 Mar 2022   Pseudo R-squ.:                 0.05114
Time:                        02:39:59   Log-Likelihood:                -135.17
converged:                       True   LL-Null:                       -142.46
Covariance Type:            nonrobust   LLR p-value:                 0.0006852
===============================================================================
                  coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------
const          -2.1348      0.649     -3.290      0.001      -3.407      -0.863
age-at-diag     0.0326      0.011      2.848      0.004       0.010       0.055
ELMO3           0.3846      0.150      2.570      0.010       0.091       0.678
===============================================================================
"""

In [7]:
scaler = clf.steps[0][1]
fn = list(scaler.feature_names_in_)
for n in missing_feature_names:
    idx = fn.index(n)
    expression_data[n] = pd.Series([scaler.mean_[idx]]*len(expression_data), dtype=np.float_)
    
Y = clf.predict(expression_data[feature_names])
print(Y)

#survival_data = pd.DataFrame.from_records(
#    [compare_survival(
#        expression_data[['death', 'over-all-survival']].to_records(index=False),
#        Y
#    )
#     for gene in genes],
#    columns=['chi-square','p-value'],
#    index=genes
#)
#survival_data.sort_values('p-value', inplace=True)
#fdr_correction = multipletests(survival_data['p-value'], method='fdr_tsbky')
#survival_data['FDR'] = fdr_correction[1]
#survival_data['reject'] = fdr_correction[0]
#survival_data

['E' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E'
 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E'
 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E'
 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E'
 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E'
 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E'
 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E'
 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E'
 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E'
 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E'
 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E'
 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E' 'E']
